In [29]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [30]:
v = '7-1'

In [31]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.add_probs'
c_model = 'deberta-v3-small.flag-context4-2-d-oof.add_probs'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [32]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [33]:
# d = pd.DataFrame(gezi.batch2list(xc))
# dg = d.groupby('pred_id')
# max_preds = dg['pred'].max()
# min_preds = dg['pred'].min()
# max_cls_preds = dg['cls_pred'].max()
# min_cls_preds = dg['cls_pred'].min()

In [34]:
# dg = df_train.groupby('id')['source'].apply(list).reset_index(name=sources)
# dmg = df_train[df_train.cell_type=='markdown'].groupby('id')['source'].apply(list).reset_index(name=sources)
# dcg = df_train[df_train.cell_type=='code'].groupby('id')['source'].apply(list).reset_index(name=sources)

In [35]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

ic(calc_metric(xp, 'cls_pred'))
ic(calc_metric(xc, 'reg_pred'))
ic(calc_metric(xc, 'cls_pred'))
ic(calc_metric(xpc, 'cls_pred'))

[08/10/22 21:10:59] 3204525517.py:8 in <module>
                    calc_metric(xp, 'cls_pred'): 0.8940525259130645
[08/10/22 21:11:06] 3204525517.py:9 in <module>
                    calc_metric(xc, 'reg_pred'): 0.9135491516384352
[08/10/22 21:11:14] 3204525517.py:10 in <module>
                    calc_metric(xc, 'cls_pred'): 0.9141187361031606
[08/10/22 21:11:22] 3204525517.py:11 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9011719210628208


0.9011719210628208

In [36]:
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  # if prob.max() < 10:
  if prob.max() < 0.8:
    marks.append(1)
    preds.append(pc_preds.get(cid, pred))
    probs.append(pc_probs.get(cid, prob))
    cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  else:
    marks.append(0)
    preds.append(pred)
    probs.append(prob)
    cls_preds.append(cls_pred)
    
xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())
ic(calc_metric(xpc, 'cls_pred'))

[08/10/22 21:11:30] 2337548337.py:27 in <module>
                    xpc['mark'].mean(): 0.5019261406823974
[08/10/22 21:11:38] 2337548337.py:28 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9020743859652581


0.9020743859652581

In [37]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=None):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  # topn = topn or len(idxes)
  topn = topn or 10
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
        x[f'top_rank_{i}'] = idxes[i]
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  diff_feats(x, 'cls_rank_pred', 'rank_pred', 'cls_rank')
  return x

def gen_cfeat(x_):
  topn = min(10, len(x_['cls_pred_ori']))
  x = {}
  keys = [
    # 'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    ]
  x = {k: x_[k] for k in keys}
  x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  x['reg_rank_pred'] = x['reg_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
        x[f'tpop_rank_{i}']  = idxes[i] 
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  diff_feats(x, 'cls_rank_pred', 'reg_rank_pred', 'cls_reg_rank')
  diff_feats(x, 'cls_rank_pred', 'cls2_rank_pred', 'cls_cls2_rank')
  diff_feats(x, 'cls2_rank_pred', 'reg_rank_pred', 'cls2_reg_rank')
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x, topn=3) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]

  feats = []
  for i in tqdm(range(len(p_feats)), desc='merge_feats'):
    fe1 = p_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')
    
    diff_feats(fe, fe1['rank_pred'], fec['reg_rank_pred'], 'pc_cls_rank')
    diff_feats(fe, fe1['cls_rank_pred'], fec['reg_rank_pred'], 'pc_cls_reg_rank')
    diff_feats(fe, fepc['rank_pred'], fec['reg_rank_pred'], 'pcc_cls_rank')
    diff_feats(fe, fepc['cls_rank_pred'], fec['reg_rank_pred'], 'pcc_cls_reg_rank')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)

    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [38]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [39]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [40]:
gezi.set_fold(df, 5, 'ancestor_id')

In [41]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

159

In [42]:
df['cls_label'] = df.rel_rank.apply(lambda x: int(x * 100))
LABEL_COL = 'rel_rank'

In [43]:
df.cls_label.max()

99

In [44]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [45]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [46]:
from catboost import CatBoostRegressor, CatBoostRanker, CatBoostClassifier
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              # 'reg_lambda': 7.960622217848342e-07, 
              'l2_leaf_reg': 3,
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              # 'max_depth': 10, 
              # 'max_depth': 8,
              'max_depth': 6,
              # 'max_depth': 5,
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [47]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
ic(cbt_score)

0:	learn: 0.2523023	test: 0.2523023	test1: 0.2522493	best: 0.2522493 (0)	total: 48.3ms	remaining: 8m 3s
100:	learn: 0.0656261	test: 0.0656261	test1: 0.0668349	best: 0.0668349 (100)	total: 3.94s	remaining: 6m 26s
200:	learn: 0.0589955	test: 0.0589955	test1: 0.0604284	best: 0.0604284 (200)	total: 7.68s	remaining: 6m 14s
300:	learn: 0.0582516	test: 0.0582516	test1: 0.0597288	best: 0.0597288 (300)	total: 11.4s	remaining: 6m 8s
400:	learn: 0.0576269	test: 0.0576269	test1: 0.0591451	best: 0.0591451 (400)	total: 15.4s	remaining: 6m 9s
500:	learn: 0.0570984	test: 0.0570984	test1: 0.0586615	best: 0.0586615 (500)	total: 19.4s	remaining: 6m 6s
600:	learn: 0.0566869	test: 0.0566869	test1: 0.0583025	best: 0.0583025 (600)	total: 23.2s	remaining: 6m 3s
700:	learn: 0.0564000	test: 0.0564000	test1: 0.0580595	best: 0.0580595 (700)	total: 27.1s	remaining: 5m 59s
800:	learn: 0.0561672	test: 0.0561672	test1: 0.0578728	best: 0.0578728 (800)	total: 31s	remaining: 5m 56s
900:	learn: 0.0560024	test: 0.0560024	

[08/10/22 21:18:44] 3198672829.py:8 in <module>
                    cbt_score: 0.9197134114106469


0.9197134114106469

In [48]:
dvalid

,code_ratio,pc_reg_diff,abs_pc_reg_diff,pc_cls_diff,abs_pc_cls_diff,pc_cls2_diff,abs_pc_cls2_diff,pc_cls_reg_diff,abs_pc_cls_reg_diff,pc_cls_cls_diff,abs_pc_cls_cls_diff,pc_cls_cls2_diff,abs_pc_cls_cls2_diff,pcc_reg_diff,abs_pcc_reg_diff,pcc_cls_diff,abs_pcc_cls_diff,pcc_cls2_diff,abs_pcc_cls2_diff,pcc_cls_reg_diff,abs_pcc_cls_reg_diff,pcc_cls_cls_diff,abs_pcc_cls_cls_diff,pcc_cls_cls2_diff,abs_pcc_cls_cls2_diff,pc_cls_rank_diff,abs_pc_cls_rank_diff,pc_cls_reg_rank_diff,abs_pc_cls_reg_rank_diff,pcc_cls_rank_diff,abs_pcc_cls_rank_diff,pcc_cls_reg_rank_diff,abs_pcc_cls_reg_rank_diff,xp_pred,xp_cls_pred,xp_rank_pred,xp_cls_rank_pred,xp_min_prob,xp_var_prob,xp_min_sim,xp_var_sim,xp_top_prob_0,xp_top_sim_0,xp_top_pred_1,xp_top_rank_1,xp_top_prob_1,xp_top_sim_1,xp_top_pred_2,xp_top_rank_2,xp_top_prob_2,xp_top_sim_2,xp_top_pred_3,xp_top_rank_3,xp_top_prob_3,xp_top_sim_3,xp_top_pred_4,xp_top_rank_4,xp_top_prob_4,xp_top_sim_4,xp_top_pred_5,xp_top_rank_5,xp_top_prob_5,xp_top_sim_5,xp_top_pred_6,xp_top_rank_6,xp_top_prob_6,xp_top_sim_6,xp_top_pred_7,xp_top_rank_7,xp_top_prob_7,xp_top_sim_7,xp_top_pred_8,xp_top_rank_8,xp_top_prob_8,xp_top_sim_8,xp_top_pred_9,xp_top_rank_9,xp_top_prob_9,xp_top_sim_9,xp_cls_diff,xp_abs_cls_diff,xp_cls_rank_diff,xp_abs_cls_rank_diff,xc_cls_pred,xc_reg_pred,xc_cls2_pred,xc_group_pos,xc_group_rank,xc_cls_rank_pred,xc_cls2_rank_pred,xc_reg_rank_pred,xc_min_prob,xc_var_prob,xc_top_prob_0,xc_top_pred_1,xc_tpop_rank_1,xc_top_prob_1,xc_top_pred_2,xc_tpop_rank_2,xc_top_prob_2,xc_top_pred_3,xc_tpop_rank_3,xc_top_prob_3,xc_top_pred_4,xc_tpop_rank_4,xc_top_prob_4,xc_top_pred_5,xc_tpop_rank_5,xc_top_prob_5,xc_top_pred_6,xc_tpop_rank_6,xc_top_prob_6,xc_top_pred_7,xc_tpop_rank_7,xc_top_prob_7,xc_top_pred_8,xc_tpop_rank_8,xc_top_prob_8,xc_top_pred_9,xc_tpop_rank_9,xc_top_prob_9,xc_cls_reg_diff,xc_abs_cls_reg_diff,xc_cls_cls2_diff,xc_abs_cls_cls2_diff,xc_cls2_reg_diff,xc_abs_cls2_reg_diff,xc_cls_reg_rank_diff,xc_abs_cls_reg_rank_diff,xc_cls_cls2_rank_diff,xc_abs_cls_cls2_rank_diff,xc_cls2_reg_rank_diff,xc_abs_cls2_reg_rank_diff,xpc_pred,xpc_cls_pred,xpc_rank_pred,xpc_cls_rank_pred,xpc_min_prob,xpc_var_prob,xpc_min_sim,xpc_var_sim,xpc_top_prob_0,xpc_top_sim_0,xpc_top_pred_1,xpc_top_rank_1,xpc_top_prob_1,xpc_top_sim_1,xpc_top_pred_2,xpc_top_rank_2,xpc_top_prob_2,xpc_top_sim_2,xpc_cls_diff,xpc_abs_cls_diff,xpc_cls_rank_diff,xpc_abs_cls_rank_diff,xpc_mark,id,cell_id,cid,n_words,n_code_cell,n_cell,ancestor_id,rel_rank,fold,cls_label,cbt_pred
54,0.5895,-0.0030,0.0030,-0.0004,0.0004,0.0018,0.0018,-0.0008,0.0008,0.0019,0.0019,0.0041,0.0041,-0.0030,0.0030,-0.0004,0.0004,0.0018,0.0018,-0.0008,0.0008,0.0019,0.0019,0.0041,0.0041,-10.4868,10.4868,-10.3585,10.3585,-10.4868,10.4868,-10.3585,10.3585,0.2368,0.2391,13.0000,13.1284,0.0000,0.0160,0.3837,0.0085,0.9640,0.7873,0.2544,14,0.0204,0.7026,0.5351,30,0.0038,0.6660,0.3070,17,0.0016,0.6462,0.3772,21,0.0015,0.6456,0.1491,8,0.0014,0.6436,0.4825,27,0.0014,0.6435,0.1667,9,0.0010,0.6366,0.0263,1,0.0008,0.6302,0.2895,16,0.0006,0.6243,0.0023,0.0023,0.1284,0.1284,0.2372,0.2399,0.2350,1,1,23.2199,23.0000,23.4868,0.0000,0.0078,0.8823,0.2550,25,0.0998,0.2850,28,0.0018,0.2450,24,0.0018,0.2150,21,0.0016,0.2650,26,0.0009,0.2750,27,0.0008,0.1550,15,0.0008,0.4150,41,0.0007,0.3350,33,0.0007,-0.0027,0.0027,0.0022,0.0022,-0.0049,0.0049,-0.2669,0.2669,0.2199,0.2199,-0.4868,0.4868,0.2368,0.2391,13.0000,13.1284,0.0000,0.0160,0.3837,0.0085,0.9640,0.7873,0.2544,14,0.0204,0.7026,0.5351,30,0.0038,0.6660,0.0023,0.0023,0.1284,0.1284,0,001308991e0c5e,05915708,001308991e0c5e\t05915708,22,56,95,c5fbea8e,0.2368,0,23,0.2406
55,0.5895,-0.0113,0.0113,-0.0117,0.0117,-0.0018,0.0018,-0.0091,0.0091,-0.0095,0.0095,0.0003,0.0003,-0.0113,0.0113,-0.0117,0.0117,-0.0018,0.0018,-0.0091,0.0091,-0.0095,0.0095,0.0003,0.0003,-33.9414,33.9414,-33.8165,33.8165,-33.9414,33.9414,-33.8165,33.8165,0.7632,0.7653,43.0000,43.1249,0.0000,0.0134,0.3038,0.0060,0.8777,0.7252,0.7807,44,0.1110,0.6798,0.7982,45,0.0098,0.6266,0.8333,47,0.0004,0.5554,0.86

In [49]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [50]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          # 'num_leaves': 32,
          # 'num_leaves': 24,
          'num_leaves': 12,
          # 'min_data_in_leaf': 300,
          'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

In [51]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                20000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=200)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36575
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 159
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.0667104	valid_1's l1: 0.0679336
[200]	training's l1: 0.0594694	valid_1's l1: 0.0608451
[300]	training's l1: 0.0584455	valid_1's l1: 0.0598846
[400]	training's l1: 0.0575776	valid_1's l1: 0.0590619
[500]	training's l1: 0.0569931	valid_1's l1: 0.0585394
[600]	training's l1: 0.0566692	valid_1's l1: 0.0582526
[700]	training's l1: 0.0564173	valid_1's l1: 0.0580325
[800]	training's l1: 0.0562509	valid_1's l1: 0.0578877
[900]	training's l1: 0.0561139	valid_1's l1: 0.0577765
[1000]	training's l1: 0.0560273	valid_1's l1: 0.0577094
[1100]	training's l1: 0.0559325	valid_1

[08/10/22 21:23:14] 1296901836.py:12 in <module>
                    lgb_score: 0.9200069806084826


0.9200069806084826

In [52]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [53]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir(f'../working/trees{v}')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees{v}/{fold}.cbt')
  dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    20000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=200)
  lgb_model.save_model(f'../working/trees{v}/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cbt_pred.values * 0.5 + dvalid.lgb_pred.values * 0.5)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/10/22 21:23:23] 134356872.py:21 in <module>
                    cbt_score: 0.9197134114106469
[08/10/22 21:23:49] 134356872.py:36 in <module>
                    lgb_score: 0.9200069806084826
[08/10/22 21:23:51] 134356872.py:38 in <module>
                    score: 0.9201324981206255
[08/10/22 21:23:51] 134356872.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9197134114106469
                    np.asarray(lgb_scores).mean(): 0.9200069806084826
                    np.asarray(scores).mean(): 0.9201324981206255


0:	learn: 0.2522899	test: 0.2522899	test1: 0.2521740	best: 0.2521740 (0)	total: 54.6ms	remaining: 9m 5s
500:	learn: 0.0571717	test: 0.0571717	test1: 0.0563018	best: 0.0563018 (500)	total: 17.5s	remaining: 5m 31s
1000:	learn: 0.0562221	test: 0.0562221	test1: 0.0555618	best: 0.0555618 (1000)	total: 35.1s	remaining: 5m 15s
1500:	learn: 0.0557530	test: 0.0557530	test1: 0.0552780	best: 0.0552780 (1500)	total: 52.8s	remaining: 4m 59s
2000:	learn: 0.0554009	test: 0.0554009	test1: 0.0551291	best: 0.0551291 (2000)	total: 1m 10s	remaining: 4m 41s
2500:	learn: 0.0550920	test: 0.0550920	test1: 0.0550349	best: 0.0550349 (2500)	total: 1m 27s	remaining: 4m 23s
3000:	learn: 0.0548411	test: 0.0548411	test1: 0.0549795	best: 0.0549795 (3000)	total: 1m 45s	remaining: 4m 5s
3500:	learn: 0.0546211	test: 0.0546211	test1: 0.0549347	best: 0.0549347 (3500)	total: 2m 2s	remaining: 3m 47s
4000:	learn: 0.0544034	test: 0.0544034	test1: 0.0548946	best: 0.0548946 (4000)	total: 2m 19s	remaining: 3m 29s
4500:	learn: 0.

[08/10/22 21:29:46] 134356872.py:21 in <module>
                    cbt_score: 0.9226709294575468


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.080387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36577
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 159
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0577029	valid_1's l1: 0.0567856
[1000]	training's l1: 0.0565838	valid_1's l1: 0.0558128
[1500]	training's l1: 0.0561936	valid_1's l1: 0.0555349
[2000]	training's l1: 0.0559532	valid_1's l1: 0.0553883
[2500]	training's l1: 0.0557758	valid_1's l1: 0.0552838
[3000]	training's l1: 0.055602	valid_1's l1: 0.0551864
[3500]	training's l1: 0.0554763	valid_1's l1: 0.0551302
[4000]	training's l1: 0.0553717	valid_1's l1: 0.0550796
[4500]	training's l1: 0.0552671	valid_1's l1: 0.0550385
[5000]	training's l1: 0.055175	valid_1's l1: 0.0550119
[5500]	training's l1: 0.0550792	v

...

[08/10/22 21:50:09] 134356872.py:36 in <module>
                    lgb_score: 0.9246888486307392
[08/10/22 21:50:11] 134356872.py:38 in <module>
                    score: 0.9249848549843661
[08/10/22 21:50:11] 134356872.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9221415341887879
                    np.asarray(lgb_scores).mean(): 0.9223065423511423
                    np.asarray(scores).mean(): 0.9224985427373306


0:	learn: 0.2522949	test: 0.2522949	test1: 0.2523827	best: 0.2523827 (0)	total: 47.8ms	remaining: 7m 57s
500:	learn: 0.0568685	test: 0.0568685	test1: 0.0583017	best: 0.0583017 (500)	total: 20.1s	remaining: 6m 20s
1000:	learn: 0.0558631	test: 0.0558631	test1: 0.0574127	best: 0.0574127 (1000)	total: 40.3s	remaining: 6m 2s
1500:	learn: 0.0553929	test: 0.0553929	test1: 0.0570973	best: 0.0570973 (1500)	total: 1m	remaining: 5m 41s
2000:	learn: 0.0550876	test: 0.0550876	test1: 0.0569553	best: 0.0569523 (1984)	total: 1m 20s	remaining: 5m 21s
2500:	learn: 0.0548067	test: 0.0548067	test1: 0.0568512	best: 0.0568512 (2500)	total: 1m 40s	remaining: 5m
3000:	learn: 0.0545569	test: 0.0545569	test1: 0.0567843	best: 0.0567843 (2999)	total: 1m 59s	remaining: 4m 39s
3500:	learn: 0.0543235	test: 0.0543235	test1: 0.0567526	best: 0.0567518 (3487)	total: 2m 19s	remaining: 4m 18s
4000:	learn: 0.0541234	test: 0.0541234	test1: 0.0567090	best: 0.0567080 (3918)	total: 2m 39s	remaining: 3m 58s
4500:	learn: 0.05392

[08/10/22 21:56:17] 134356872.py:21 in <module>
                    cbt_score: 0.9154378911330956


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.078949 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36575
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 159
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0571592	valid_1's l1: 0.0585579
[1000]	training's l1: 0.0560881	valid_1's l1: 0.0575669
[1500]	training's l1: 0.055739	valid_1's l1: 0.0572973
[2000]	training's l1: 0.0555563	valid_1's l1: 0.0571727
[2500]	training's l1: 0.0553973	valid_1's l1: 0.0570686
[3000]	training's l1: 0.0552667	valid_1's l1: 0.0569887
[3500]	training's l1: 0.0551529	valid_1's l1: 0.056936
[4000]	training's l1: 0.0550718	valid_1's l1: 0.0568921
[4500]	training's l1: 0.0550001	valid_1's l1: 0.0568583
[5000]	training's l1: 0.0549273	valid_1's l1: 0.0568179
[5500]	training's l1: 0.0548501	v

[08/10/22 22:01:16] 134356872.py:36 in <module>
                    lgb_score: 0.9153802952740407
[08/10/22 22:01:18] 134356872.py:38 in <module>
                    score: 0.9156765964044635
[08/10/22 22:01:18] 134356872.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9208008055776494
                    np.asarray(lgb_scores).mean(): 0.920921292935722
                    np.asarray(scores).mean(): 0.9211341534707571


In [54]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614